In [1]:
import cv2
import face_recognition
import numpy as np
import os
from datetime import datetime

In [2]:
path = 'FaceImages'
images = []
classNames = []
myList = os.listdir(path)
print(myList)

['Ciara.jpeg', 'Batuhan.jpeg', '.DS_Store', 'Caitriona.jpeg', 'Beyonce.jpeg', 'Rihanna.jpeg', '.ipynb_checkpoints', 'Gabby.jpeg']


In [3]:
for cls in myList:
    img = cv2.imread(f'{path}/{cls}')

    if img is None:
        print(f"Failed to load image: {cls}")
        continue
    
    images.append(img)
    classNames.append(os.path.splitext(cls)[0])
print(classNames)

Failed to load image: .DS_Store
Failed to load image: .ipynb_checkpoints
['Ciara', 'Batuhan', 'Caitriona', 'Beyonce', 'Rihanna', 'Gabby']


In [4]:
def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        if img is None:
            print("Empty image detected, skipping...")
            continue
        
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList

encodeListKnown = findEncodings(images)
print('Encoding done.')

Encoding done.


In [5]:
def markClockIn(name):
    with open('Clock_In_Sheet.csv', 'r+') as f:
        myDataList = f.readlines()
        nameList = []
        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])
            
        if name not in nameList:
            now = datetime.now()
            dtString = now.strftime('%H:%M:%S')
            f.writelines(f'\n{name}, {dtString}')

In [6]:
cap = cv2.VideoCapture(0)

In [7]:
import time

timeout = time.time() + 10  

while True:
    success, img = cap.read()
    if not success:
        break
    
    small_img = cv2.resize(img, (0, 0), None, 0.25, 0.25)
    small_img = cv2.cvtColor(small_img, cv2.COLOR_BGR2RGB)
  
    face_locations = face_recognition.face_locations(small_img)

    face_locations = [(top*4, right*4, bottom*4, left*4) for (top, right, bottom, left) in face_locations]
    
    encodeCurFrame = face_recognition.face_encodings(img, face_locations)
    
    for encodeFace, faceLoc in zip(encodeCurFrame, face_locations):
        matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
        faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
        
        matchIndex = np.argmin(faceDis)
        
        if matches[matchIndex]:
            name = classNames[matchIndex].upper()
            y1, x2, y2, x1 = faceLoc
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
            cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
            
            markClockIn(name)
    
    cv2.imshow('Webcam', img)
    
    if cv2.waitKey(1) & 0xFF == ord('q') or time.time() > timeout:
        break

cap.release()
cv2.destroyAllWindows()